In [1]:
import pandas as pd
import numpy as np

# Read the datasets
df_field = pd.read_csv('../dataset/field.csv')
df_satel = pd.read_csv('../dataset/sentinel.csv')

# Convert Date columns to datetime
df_field['Date'] = pd.to_datetime(df_field['Date'])
df_satel['Date'] = pd.to_datetime(df_satel['Date'])

In [2]:
# Convert USGS units to SI units
# Discharge: cfs to m³/s (1 cfs = 0.0283168 m³/s)
df_field['Discharge'] = df_field['Discharge'] * 0.0283168

# Height: feet to meters (1 foot = 0.3048 m)
df_field['Height'] = df_field['Height'] * 0.3048

df_field.head()

,Date,Discharge,Height,Turbidity,pH,DO,SC,Temperature,Chl-a,Phycocyanin
0,2018-11-20,509.70240,3.788664,62.8,8.2,11.8,508,6.9,0.7,0.56
1,2018-11-21,577.66272,4.011168,70.2,8.2,11.7,486,7.5,0.7,0.55
2,2018-11-22,600.31616,4.075176,69.6,8.1,11.6,474,7.9,0.7,0.54
3,2018-11-23,600.31616,4.078224,65.0,8.1,11.4,472,8.2,0.7,0.52
4,2018-11-24,603.14784,4.084320,63.3,8.1,11.4,476,7.9,0.7,0.53


In [3]:
band_cols = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'TCI_B', 'TCI_G', 'TCI_R']
aot_wvp_cols = ['AOT', 'WVP']

In [4]:
for col in band_cols:
    if col in df_satel.columns:
        df_satel[f'{col}_scaled'] = np.maximum(df_satel[col] / 10000, 0)

for col in aot_wvp_cols:
    if col in df_satel.columns:
        df_satel[f'{col}_scaled'] = df_satel[col] / 1000

In [5]:
df_satel = df_satel.drop(columns=[col for col in band_cols + aot_wvp_cols if col in df_satel.columns])

In [6]:
rename_dict = {f'{col}_scaled': col for col in band_cols + aot_wvp_cols if f'{col}_scaled' in df_satel.columns}
df_satel = df_satel.rename(columns=rename_dict)
df_satel.head()

,Date,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12,TCI_B,TCI_G,TCI_R,AOT,WVP
0,2017-04-10,0.042775,0.052880,0.078256,0.102993,0.114237,0.090991,0.095663,0.092736,0.077082,0.061807,0.034235,0.027916,0.005433,0.007997,0.010511,0.058000,0.506207
1,2017-06-09,0.053704,0.065506,0.091527,0.115390,0.128128,0.097415,0.104769,0.093351,0.081053,0.083656,0.034671,0.028664,0.006715,0.009346,0.011770,0.097544,1.867486
2,2017-06-24,0.043489,0.059528,0.081443,0.097508,0.104992,0.069905,0.076587,0.067627,0.055742,0.070095,0.029018,0.022429,0.006108,0.008322,0.009954,0.090176,1.320891
3,2017-06-29,0.063772,0.075828,0.103117,0.118605,0.133740,0.097191,0.103646,0.087453,0.082890,0.095511,0.053343,0.044068,0.007756,0.010526,0.012100,0.300032,2.859769
4,2017-08-03,0.054517,0.062940,0.081608,0.066941,0.090995,0.072792,0.080886,0.068803,0.066734,0.102755,0.043727,0.031002,0.006451,0.008340,0.006850,0.197861,2.679410


In [7]:
# Merge the datasets on Date (inner join)
merged_df = pd.merge(df_field, df_satel, on='Date', how='inner')

# Compute satellite band combination indices

- **MNDWI, NDWI:** Water body delineation and inundation.
- **GNDVI:** Vegetation/algae health (relevant for Chl-a).
- **SDDI:** Secchi disk depth.
- **NDTI:** Turbidity/total suspended matter.
- **BR:** Water clarity.
- **NDPI:** Pond/standing water (adaptable for river segments).
- **NDCI:** Chlorophyll-a concentration.

In [8]:
# MNDWI = (Green - SWIR) / (Green + SWIR) = (B3 - B11) / (B3 + B11)
merged_df['MNDWI'] = (merged_df['B3'] - merged_df['B11']) / (merged_df['B3'] + merged_df['B11'])

# GNDVI = (NIR - Green) / (NIR + Green) = (B8 - B3) / (B8 + B3)
merged_df['GNDVI'] = (merged_df['B8'] - merged_df['B3']) / (merged_df['B8'] + merged_df['B3'])

# SDDI = Log(Green/Red) = Log(B3 / B4)
merged_df['SDDI'] = np.log(merged_df['B3'] / merged_df['B4'])

# NDTI = (Red - Green) / (Red + Green) = (B4 - B3) / (B4 + B3)
merged_df['NDTI'] = (merged_df['B4'] - merged_df['B3']) / (merged_df['B4'] + merged_df['B3'])

# BR = (Blue / Red) = B2 / B4
merged_df['BR'] = merged_df['B2'] / merged_df['B4']

# NDWI = (Green - NIR) / (Green + NIR) = (B3 - B8) / (B3 + B8)
merged_df['NDWI'] = (merged_df['B3'] - merged_df['B8']) / (merged_df['B3'] + merged_df['B8'])

# NDPI = (SWIR - Green) / (SWIR + Green) = (B11 - B3) / (B11 + B3)
merged_df['NDPI'] = (merged_df['B11'] - merged_df['B3']) / (merged_df['B11'] + merged_df['B3'])

# NDCI = (RedEdge1 - Red) / (RedEdge1 + Red) = (B5 - B4) / (B5 + B4)
merged_df['NDCI'] = (merged_df['B5'] - merged_df['B4']) / (merged_df['B5'] + merged_df['B4'])

# 2BDA (2-Band Difference Algorithm) for Chlorophyll-a Proxy: B5 - B4
merged_df['2BDA_Chl'] = merged_df['B5'] - merged_df['B4']

# Red Edge / Red ratio for turbidity/sediment: B5 / B4
merged_df['RR'] = merged_df['B5'] / merged_df['B4']

In [9]:
field_cols = [col for col in df_field.columns if col != 'Date']
satel_cols = [col for col in df_satel.columns if col != 'Date']
index_cols = [col for col in merged_df.columns if col.startswith(('MNDWI', 'GNDVI', 'SDDI', 'NDTI', 'BR', 'NDWI', 'NDPI', 'NDCI', '2BDA_Chl', 'RR'))]

merged_df = merged_df[['Date'] + field_cols + satel_cols + index_cols]
merged_df.head()

,Date,Discharge,Height,Turbidity,pH,DO,SC,Temperature,Chl-a,Phycocyanin,...,MNDWI,GNDVI,SDDI,NDTI,BR,NDWI,NDPI,NDCI,2BDA_Chl,RR
0,2018-11-21,577.66272,4.011168,70.2,8.2,11.7,486,7.5,0.7,0.55,...,0.407433,-0.082268,-0.278961,0.138583,0.465241,0.082268,-0.407433,0.016434,0.003132,1.033418
1,2018-12-16,543.68256,3.849624,93.3,8.0,13.3,657,2.6,0.8,0.74,...,0.426877,-0.046317,-0.374309,0.185000,0.386335,0.046317,-0.426877,0.025896,0.004890,1.053169
2,2018-12-21,478.55392,3.624072,67.7,8.0,12.9,672,3.4,0.7,0.67,...,0.445335,-0.129644,-0.339137,0.167962,0.343961,0.129644,-0.445335,0.013995,0.002476,1.028387
3,2019-01-05,521.02912,3.767328,107.0,8.0,13.6,611,1.9,0.8,0.80,...,0.457476,-0.022581,-0.368427,0.182158,0.371143,0.022581,-0.457476,0.032999,0.006492,1.068250
4,2019-02-24,342.63328,3.218688,541.0,8.1,13.7,576,1.1,4.2,2.18,...,0.411478,0.406900,-0.478754,0.234907,0.292060,-0.406900,-0.411478,0.090299,0.018530,1.198525


In [10]:
merged_df.to_csv('../dataset/merged.csv', index=False)
print("Merged data with unit conversions and indices exported.")

Merged data with unit conversions and indices exported.
